# External Inconsistencies

In [30]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from IPython.display import display
import pyomo.environ as pyo
from pyomo.opt import SolverFactory


In [31]:
countries = {'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU',
             'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME', 'Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'GB'}
abbr_list = list(countries.values())
abbr_list = [country for country in abbr_list if country != 'CH']

In [32]:
# def calculate_sigma:
# load_gen_data = {}
# sigma = {}
# for abbr in abbr_list:
#     df = pd.read_csv(f"../Data Sources/output/Sigma/Generation/{abbr}.csv").iloc[:,1:]
#     df = pd.concat([df, pd.read_csv(f"../Data Sources/output/Sigma/Load/{abbr}.csv").iloc[:,1:]], axis=1)
#     sigma[abbr] = df

# sigma["transmission_data"] = pd.read_csv(f"../Data Sources/output/Sigma/Transmission/all_transmissions.csv").iloc[:,1:]
generation_dic = {}
for abbr in abbr_list:
    df = pd.read_csv(f"../Data Sources/output/Sigma/Generation/{abbr}.csv").iloc[:,1:]
    df = df.drop(['Other'], axis=1, errors='ignore')
    columns = [column for column in df.columns.values if "Hydro" in column]
    df['Hydro'] = df.loc[:,columns].sum(axis=1)
    df=df.drop(columns,axis=1)
    columns = [column for column in df.columns.values if ("Coal" in column) or ("coal" in column)]
    df['Coal'] = df.loc[:, columns].sum(axis=1)
    df=df.drop(columns, axis=1)
    columns = [column for column in df.columns.values if "Wind" in column]
    df['Wind'] = df.loc[:, columns].sum(axis=1)
    df=df.drop(columns, axis=1)
    generation_dic[abbr] = df



In [33]:
country_index = abbr_list
time_index = np.arange(8760, dtype=int)

generation_index = {}

for abbr, df in generation_dic.items():
    generation_index[abbr] = [x for x in df.columns.values]

generation_fuels = np.array(list(set([item for sublist in generation_index.values() for item in sublist])))
generation_fuels.sort()


In [34]:
for abbr,df in generation_dic.items():
    for fuel in generation_fuels:
        if not fuel in df.columns.values:
            df[fuel] = 0 

In [35]:
df = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/Generation Capacity/DE.csv', low_memory=False).iloc[:-1, :]
columns = df['Production Type'].values
index = abbr_list
capacity = pd.DataFrame(index=index,columns = columns)
for abbr in abbr_list:
    df = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/Generation Capacity/{abbr}.csv', low_memory=False).iloc[:-1, :]
    for index,row in enumerate(df.values):
        capacity.loc[abbr][row[0]] = row[1]
capacity = capacity.replace(['n/e',np.nan],0)
for column in capacity.columns.values:
    capacity[column] = pd.to_numeric(capacity[column])
columns = [column for column in capacity.columns.values if "Hydro" in column]
capacity['Hydro'] = capacity.loc[:, columns].sum(axis=1)
capacity = capacity.drop(columns, axis=1)
columns = [column for column in capacity.columns.values if ("Coal" in column) or ("coal" in column)]
capacity['Coal'] = capacity.loc[:, columns].sum(axis=1)
capacity = capacity.drop(columns, axis=1)
columns = [column for column in capacity.columns.values if "Wind" in column]
capacity['Wind'] = capacity.loc[:, columns].sum(axis=1)
capacity = capacity.drop(columns, axis=1)

capacity_total = {}
for abbr in abbr_list:
    capacity_total[abbr] = {}
    # for item in generation_index[abbr]:
    for item in generation_fuels:
        capacity_total[abbr][item] = capacity.loc[abbr][item]


In [36]:
generation_total = {}
em = pd.read_excel('../Data Sources/External sources.xlsx',
                   sheet_name='EUROSTAT',index_col=0)
for abbr in abbr_list:
    generation_total[abbr] = {}
    # for item in generation_index[abbr]:
    for item in generation_fuels:
        generation_total[abbr][item] = em[item][abbr]*1000



In [37]:
model = pyo.ConcreteModel()

model.country_index = pyo.Set(initialize=country_index)
model.time_index = pyo.Set(initialize=time_index)
model.generation_fuels = pyo.Set(initialize=generation_fuels)

model.alpha = pyo.Var(model.country_index, model.generation_fuels, model.time_index, bounds=(0.0, None))
model.beta = pyo.Var(model.country_index, model.generation_fuels, model.time_index, bounds=(0.0, None))
model.delta = pyo.Var(model.country_index, model.generation_fuels, model.time_index, bounds=(0.0, None))

print('VARIABLES DECLARED')

def balance_rule1(model, country, fuel, time): 
    return model.alpha[country, fuel, time]*generation_dic[country][fuel][time] + model.beta[country, fuel, time] - model.delta[country, fuel, time] <= capacity_total[country][fuel]
model.balance_rule1 = pyo.Constraint(model.country_index, model.generation_fuels, model.time_index, rule=balance_rule1)

def balance_rule2(model, country, fuel):   
    return sum(model.alpha[country, fuel, time]*generation_dic[country][fuel][time] + model.beta[country, fuel, time] - model.delta[country, fuel, time] for time in time_index) == generation_total[country][fuel]
model.balance_rule2 = pyo.Constraint(model.country_index, model.generation_fuels, rule=balance_rule2)

def ObjRule(model):
    return sum(model.beta[country, fuel, time] + model.delta[country, fuel, time] for country in country_index for fuel in generation_index[country] for time in time_index)

model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
opt = SolverFactory("gurobi", solver_io="python")
opt.solve(model)
print('OPTIMIZATION COMPLETED')


VARIABLES DECLARED
Academic license - for non-commercial use only - expires 2021-12-14
Using license file C:\Users\lovin\gurobi.lic
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
